In [ ]:
import os
import mlflow
# from sklearn.metrics 
from dotenv import load_dotenv
load_dotenv(override=True)

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
YOUR_NAME = "Slava"
assert YOUR_NAME, "enter your name"

EXPERIMENT_NAME = f"test_connection_experiment_{YOUR_NAME}"
RUN_NAME = f"test_connection_run"

METRIC_NAME = "test_metric"
METRIC_VALUE = 0

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.log_metric(METRIC_NAME, METRIC_VALUE)


In [6]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
run = mlflow.get_run(run_id)

assert "active" == experiment.lifecycle_stage
assert mlflow.get_run(run_id)
assert METRIC_VALUE == run.data.metrics[METRIC_NAME]

In [2]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import joblib
import mlflow
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, log_loss
from dotenv import load_dotenv
load_dotenv(override=True)
np.random.seed(43)

n_samples = 1000

X = pd.DataFrame({
    "feature_1": np.random.randn(n_samples),
    "feature_2": np.random.randn(n_samples),
    "feature_3": np.random.randn(n_samples),
})


y = (
    (X["feature_1"] + 0.5 * X["feature_2"] - 0.3 * X["feature_3"]) > 0
).astype(int)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = CatBoostClassifier(
    iterations=200,
    depth=6,
    learning_rate=0.1,
    loss_function="Logloss",
    verbose=False
)

model.fit(X_train, y_train)

prediction = model.predict(X_test)
probas = model.predict_proba(X_test)

model_path = "catboost_classifier.pkl"
X_train.to_csv("train_data.csv", index=False)
X_test.to_csv("test_data.csv", index=False)


joblib.dump(model, model_path)

['catboost_classifier.pkl']

In [3]:
metrics = {}

_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize="all").ravel()
auc = roc_auc_score(y_test, probas[:, 1])
precision = precision_score(y_test, prediction)
recall = recall_score(y_test, prediction)
f1 = f1_score(y_test, prediction)
logloss = log_loss(y_test, prediction)

metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

In [4]:
class CatboostModelProba(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        super().__init__()
        self._model = model

    def predict(self, context, model_input):
        predictions = self._model.predict(model_input)
        predictions = np.sqrt(predictions)

        return predictions

In [5]:
experiment_id = "0"
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
EXPERIMENT_NAME = "Slavuta"
RUN_NAME="model_for_registry_4"
REGISTRY_MODEL_NAME="test_model_slavuta"

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

pip_requirements = "requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {"model_type": "daily"}

try:
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
except AttributeError:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    for k, v in metrics.items():
        mlflow.log_metric(k, v)

    model_info = mlflow.catboost.log_model(
        registered_model_name=REGISTRY_MODEL_NAME,
        cb_model=model,
        artifact_path="models",
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        metadata=metadata,
        await_registration_for=60
    )

Registered model 'test_model_slavuta' already exists. Creating a new version of this model...
2026/02/05 13:54:16 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: test_model_slavuta, version 5
Created version '5' of model 'test_model_slavuta'.


In [6]:
loaded_model = mlflow.catboost.load_model(model_uri=model_info.model_uri)
model_predictions = loaded_model.predict(X_test)

assert model_predictions.dtype == int

print(model_predictions[:10])

[0 0 1 1 1 0 1 0 1 0]


In [ ]:
# Task to log a new model

import os
import mlflow
from dotenv import load_dotenv
load_dotenv(override=True)

EXPERIMENT_NAME = "churn_nikolaistepanov_myown"
RUN_NAME = "model_0_registry_myonwn"
REGISTRY_MODEL_NAME = "churn_model_nikolaistepanov_model_myown"
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

pip_requirements = "requirements.txt"
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {"model_type": "daily"}

try:
    experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
except AttributeError:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)


with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    model = CatBoostClassifier(verbose=False)
    model.fit(X_train, y_train)
    
    prediction = model.predict(X_test)
    probas = model.predict_proba(X_test)

    metrics = {}

    _, err1, _, err2 = confusion_matrix(y_test, prediction, normalize="all").ravel()
    auc = roc_auc_score(y_test, probas[:, 1])
    precision = precision_score(y_test, prediction)
    recall = recall_score(y_test, prediction)
    f1 = f1_score(y_test, prediction)
    logloss = log_loss(y_test, prediction)

    metrics["err1"] = err1
    metrics["err2"] = err2
    metrics["auc"] = auc
    metrics["precision"] = precision
    metrics["recall"] = recall
    metrics["f1"] = f1
    metrics["logloss"] = logloss

    mlflow.log_metrics(metrics)

    model_info = mlflow.catboost.log_model(
        registered_model_name=REGISTRY_MODEL_NAME,
        cb_model = model,
        artifact_path="models",
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        metadata=metadata,
    )


Successfully registered model 'churn_model_nikolaistepanov_model_myown'.
2026/02/05 13:54:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: churn_model_nikolaistepanov_model_myown, version 1
Created version '1' of model 'churn_model_nikolaistepanov_model_myown'.


In [7]:
# Getting results of experiments

import os
import mlflow
from dotenv import load_dotenv
load_dotenv(override=True)


TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

tracking_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"
registry_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"

client = mlflow.MlflowClient(tracking_uri=tracking_uri, registry_uri=registry_uri)



EXPERIMENT_NAME = "churn_nikolaistepanov_myown"

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

experiment_runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
).sort_values(by="start_time", ascending=False)

AttributeError: 'NoneType' object has no attribute 'experiment_id'